##### Copyright 2020 The Cirq Developers

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Quantum Computing Service reservation utility

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.example.org/cirq/tutorials/google/reservations"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on QuantumLib</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/quantumlib/Cirq/blob/master/docs/tutorials/google/reservations.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/quantumlib/Cirq/blob/master/docs/tutorials/google/reservations.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/Cirq/docs/tutorials/google/reservations.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

In [2]:
try:
    import cirq
except ImportError:
    print("installing cirq...")
    !pip install --quiet cirq
    print("installed cirq.")

This Colab notebook provides canned interactions with Cirq to manage a project's reservations on the Quantum Computing Service.

For information on how to download a Colab notebook from GitHub, see [these instructions](colab.ipynb).  You can also [view this file](https://github.com/quantumlib/Cirq/blob/master/docs/tutorials/google/reservations.ipynb) or download the [raw content](https://raw.githubusercontent.com/quantumlib/Cirq/master/docs/tutorials/google/reservations.ipynb) of this content from Github.

## Configure

Choose the project to manage, autheticate, and install the necessary tools.

**Note: If you are running on Jupyter notebook, please don't forget to change `project_id`, `processor_id` and `time_zone` to your preferences** 

In [3]:
def authenticate_user():
  """Runs the user through the Colab OAuth process.
  
  Checks for Google Application Default Credentials and runs interactive login 
  if the notebook is executed in Colab. In case the notebook is executed in Jupyter notebook
  or other IPython runtimes, no interactive login is provided, it is assumed that the 
  `GOOGLE_APPLICATION_CREDENTIALS` env var is set or `gcloud auth application-default login`
  was executed already.
  
  For more information on using Application Default Credentials see 
  https://cloud.google.com/docs/authentication/production
  """
  in_colab = False
  try:
     from IPython import get_ipython
     in_colab = 'google.colab' in str(get_ipython())
  except: 
     # Notebook is not executed within IPython. Assuming external authentication.
     return 

  if in_colab: 
      from google.colab import auth      
      print("Getting OAuth2 credentials.")
      print("Press enter after entering the verification code.")
      auth.authenticate_user(clear_output=False)
      print("Authentication complete.")
  else: 
      print("Notebook is not executed with Colab, assuming Application Default Credentials are setup.") 

authenticate_user()

Notebook is not executed with Colab, assuming Application Default Credentials are setup.


In [15]:
import cirq.google
from cirq.google.engine.client.quantum_v1alpha1.gapic import enums

# The Google Cloud Project id to use.
project_id = '' #@param {type:"string"}
# The processor to view.
processor_id = 'mcgee' #@param ['rainbow', 'pacific', 'mcgee']
# The local time zone.
time_zone = 'America/Los_Angeles'  #@param {type:"string"}

if project_id == '':
    import os 
    if 'GOOGLE_CLOUD_PROJECT' not in os.environ:
        raise Exception("Please setup project_id in this cell or set the `GOOGLE_CLOUD_PROJECT` env var to your project id.")
    project_id = os.environ['GOOGLE_CLOUD_PROJECT']
else: 
    os.environ['GOOGLE_CLOUD_PROJECT'] = project_id



# Create an Engine object to use, providing the project id and the args
try: 
    engine = cirq.google.get_engine()
    engine.list_processors()
    print(f"Successful authentication using project {project_id}!")
except DefaultCredentialsError as err: 
    print("Could not authenticate to Google Quantum Computing Service.")
    print(" Tips: If you are using Colab: make sure the previous cell was executed successfully.")
    print("       If this notebook is not in Colab (e.g. Jupyter notebook), make sure gcloud is installed and `gcloud auth application-default login` was executed.")
    print()
    print("Error message:")
    print(err)
except PermissionDenied as err:
    print(f"While you are authenticated to Google Cloud it seems the project '{project_id}' does not exist or does not have the Quantum Engine API enabled.")
    print("Error message:")
    print(err)
processor = engine.get_processor(processor_id)

import datetime
import pytz
tz = pytz.timezone(time_zone)
now = tz.localize(datetime.datetime.now())

def date_string(timestamp):
  if timestamp.seconds < 0:
    return 'Beginning of time'
  if timestamp.seconds > 4771848621:
    return 'End of time'
  time = datetime.datetime.fromtimestamp(timestamp.seconds).astimezone(tz)
  return time.strftime('%Y-%m-%d %H:%M:%S')
  
def delta(start, end):
  if start.seconds < 0 or end.seconds > 5000000000:
    return "∞"
  return "{} hrs".format((end.seconds - start.seconds) / (60 * 60))

def time_slot_string(time_slot):
  start = date_string(time_slot.start_time)
  end = date_string(time_slot.end_time)
  slot_type = cirq.google.engine.client.quantum_v1alpha1.types.QuantumTimeSlot.TimeSlotType.Name(time_slot.slot_type)
  slot_string = "{} to {} ({}) - {}".format(start, end, delta(time_slot.start_time, time_slot.end_time), slot_type)
  if time_slot.HasField('reservation_config'):
    return "{} for {}".format(slot_string, time_slot.reservation_config.project_id) 
  if time_slot.HasField('maintenance_config'):
    return "{} {} - {}".format(slot_string, time_slot.maintenance_config.title, time_slot.maintenance_config.description) 
  return slot_string

def reservation_string(reservation):
  start = date_string(reservation.start_time)
  end = date_string(reservation.end_time)
  id = reservation.name.split('/reservations/')[1]
  reservation_string = "{} to {} ({}) - {}".format(start, end, delta(reservation.start_time, reservation.end_time), id)
  if len(reservation.whitelisted_users) > 0:
    return "{} {}".format(reservation_string, reservation.whitelisted_users)
  return reservation_string 

print("============================")
print("Runtime setup completed")

Successful authentication using project balintp-gcp-lab!
Runtime setup completed


## Checkout the upcoming schedule

In [4]:
#@title
schedule = processor.get_schedule()
for s in schedule:
  print(time_slot_string(s))

2020-10-18 00:00:00 to 2020-11-06 14:00:00 (471.0 hrs) - OPEN_SWIM
2020-11-06 14:00:00 to End of time (∞) - UNALLOCATED


## Find available time

In [5]:
#@title
schedule = processor.get_schedule()
unallocated = list(filter(lambda t: t.slot_type == enums.QuantumTimeSlot.TimeSlotType.UNALLOCATED, schedule))
for s in unallocated:
  print(time_slot_string(s))

if len(unallocated) == 0:
  print("No available time slots")

2020-11-06 14:00:00 to End of time (∞) - UNALLOCATED


## List upcoming reservations for the project

In [6]:
#@title
reservations = processor.list_reservations()

for r in reservations:
  print(reservation_string(r))
else: 
  print(f"No reservations for project {project_id}")

No reservations for project <xyz>


## Reserve time

In [7]:
#@markdown Create a new reservation for the given start date and time with the given duration in hours.
start_date_picker = "2020-11-06" #@param {type:"date"}
start_time = "15" #@param [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23]
hours =  1 #@param {type:"integer"}
#@markdown Comma-separated email addresses of any additional users to explicitly whitelist for the reservation.
whitelist_user_emails = "" #@param {type:"string"}

start_time_naive = datetime.datetime.strptime("{} {}".format(start_date_picker, start_time), '%Y-%m-%d %H')
start_time = tz.localize(start_time_naive)
end_time = start_time + datetime.timedelta(hours=hours)

print(reservation_string(processor.create_reservation(start_time=start_time,
                                   end_time=end_time,
                                   whitelisted_users=[e.strip() for e in whitelist_user_emails.split(',') if e])))

## Update reservation time

In [8]:
#@markdown Update the deatils of an existing reservation. _You can find the `reservation_id` by listing your reservations or checking the output when you create a new reservation._
reservation_id = "" #@param {type:"string"}
start_date_picker = "2020-03-27" #@param {type:"date"}
start_time = "12" #@param [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23]
hours =  2#@param {type:"integer"}

start_time_naive = datetime.datetime.strptime("{} {}".format(start_date_picker, start_time), '%Y-%m-%d %H')
start_time = tz.localize(start_time_naive)
end_time = start_time + datetime.timedelta(hours=hours)

print(reservation_string(processor.update_reservation(reservation_id=reservation_id,
                                   start_time=start_time,
                                   end_time=end_time)))

## Update reservation whitelisted users

In [9]:
#@markdown Update the deatils of an existing reservation. _You can find the `reservation_id` by listing your reservations or checking the output when you create a new reservation._
reservation_id = "" #@param {type:"string"}
#@markdown Comma-separated email addresses of any additional users to explicitly whitelist for the reservation.
whitelisted_user_emails = "" #@param {type:"string"}

print(reservation_string(processor.update_reservation(reservation_id=reservation_id,
                                   whitelisted_users=[e.strip() for e in whitelisted_user_emails.split(',') if e])))

## Remove reservation

In [10]:
#@markdown Delete a specific reservation as long as it is outside the schedule freeze. Inside the schedule freeze period reservations are cancelled instead.
reservation_id = "" #@param {type:"string"}

processor.remove_reservation(reservation_id)